<a href="https://colab.research.google.com/github/preetham3216/info5731-fall2021/blob/main/INFO5731_Assignment_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(20 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect all User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the abstracts research papers by using the query [smart health](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 10,000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter with the following requirements: Location (Texas), Time frame (2021-01-01 to 2021-09-01). 


In [ ]:
# import all the necessary modules
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import json

url='https://www.imdb.com/title/tt9376612/reviews?ref_=tt_urv'

driver = webdriver.Chrome(executable_path=r'C:\Users\preet\Downloads\chromedriver.exe')
driver.get(url)

while True:
 try:
  driver.find_element_by_css_selector('#load-more-cc-objects').click()
 except:
  break
  
soup = BeautifulSoup(driver.page_source, features="html.parser")
reviews = soup.findAll('div', attrs={'class': 'text show-more__control'})
listOfReviews = [review.get_text().replace('\n', "") for review in reviews]
df = pd.DataFrame({'User Reviews': listOfReviews})
df.to_csv('userReviews.csv')
df.head()

,User Reviews
0,After 10 years of almost every movie being.arm...
1,Perfect Fantasy film to watch with full family...
2,Keeping it short. This movie had it all. Great...
3,Brought to you by the Truth Tellers.Film is gr...
4,Haven't been much of a Marvel guy even with th...


In [ ]:
!pip install textblob

# **Question 2**

(30 points) Here is a [legal case](https://github.com/unt-iialab/info5731-fall2021/blob/main/assignments/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



(1) Basic feature extraction using text data

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

(2) Basic Text Pre-processing of text data

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

(3) Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above.


(4) Advance Text Processing

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.


In [ ]:
# Write your code here
import pandas as pd 
import csv 
url='https://raw.githubusercontent.com/unt-iialab/info5731-fall2021/main/assignments/01-05-1%20%20Adams%20v%20Tanner.txt' # url
train = pd.read_csv(url,error_bad_lines=False,header=0,quoting=csv.QUOTE_NONE,warn_bad_lines=False) # conversion
train.columns=['data']

print(len(train)) 
#no. of sentences

# word count
train['word_count'] = train['data'].apply(lambda x: len(str(x).split(" ")))
print(train[['data','word_count']].head()) 

# total word count 
totwordcount=train['word_count'].sum()
print(totwordcount)


train['char_count'] = train['data'].str.len() 
print(train[['data','char_count']].head())


def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+1))

train['avg_word'] = train['data'].apply(lambda x: avg_word(x))
print(train[['data','avg_word']].head())


from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

train['stopwords'] = train['data'].apply(lambda x: len([x for x in x.split() if x in stop]))
print(train[['data','stopwords']].head())





train['specialchar'] = train['data'].apply(lambda x: len([x for x in x.split() if x.startswith('#' or '@' or '.')])) #included #,@,.
print(train[['data','specialchar']].head())
    
  
train['numerics'] = train['data'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
print(train[['data','numerics']].head())


train['upper'] = train['data'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
print(train[['data','upper']].head())


train['data'] = train['data'].apply(lambda x: " ".join(x.lower() for x in x.split()))
print(train['data'].head())


train['data'] = train['data'].str.replace('[^\w\s]','')
print(train['data'].head())


from nltk.corpus import stopwords
stop = stopwords.words('english')
train['data'] = train['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
print(train['data'].head())


freq = pd.Series(' '.join(train['data']).split()).value_counts()[:10]
freq = list(freq.index)
train['data'] = train['data'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
print(train['data'].head())


freq = pd.Series(' '.join(train['data']).split()).value_counts()[-10:]
freq = list(freq.index)
train['data'] = train['data'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
print(train['data'].head())


from textblob import TextBlob
print(train['data'][:5].apply(lambda x: str(TextBlob(x).correct())))


#tokenization
import nltk
nltk.download('punkt')
print(TextBlob(train['data'][1]).words)


#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
print(train['data'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()])))


#lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')

train['data'] = train['data'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print(train['data'].head())    
#df to csv conversion
train.to_csv('op.csv')






train_new = pd.read_csv(r'C:\Users\preet\OneDrive\Desktop\op.csv',index_col=0)
train_new = train_new.dropna()  
train_new.head()





106
                        data  word_count
0  Supreme Court of Alabama.           4
1                      ADAMS           1
2                         v.           1
3         TANNER AND HORTON.           3
4                   Synopsis           1
358
                        data  char_count
0  Supreme Court of Alabama.          25
1                      ADAMS           5
2                         v.           2
3         TANNER AND HORTON.          18
4                   Synopsis           8
                        data  avg_word
0  Supreme Court of Alabama.       4.4
1                      ADAMS       2.5
2                         v.       1.0
3         TANNER AND HORTON.       4.0
4                   Synopsis       4.0
                        data  stopwords
0  Supreme Court of Alabama.          1
1                      ADAMS          0
2                         v.          0
3         TANNER AND HORTON.          0
4                   Synopsis          0
                        da

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\preet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-18-55e6498b890d>:61: FutureWarning: The default value of regex will change from True to False in a future version.
  train['data'] = train['data'].str.replace('[^\w\s]','')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\preet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\preet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,data,word_count,char_count,avg_word,stopwords,specialchar,numerics,upper
0,supreme alabama,4,25,4.4,1,0,0,0
1,adam,1,5,2.5,0,0,0,1
2,tanner horton,3,18,4.0,0,0,0,3
3,synopsis,1,8,4.0,0,0,0,0
4,writ error sumter,9,45,3.7,4,0,0,1


In [ ]:
# code for term frequency 
tf = (train_new['data']).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf.columns = ['words','tf']
print(tf)

        words   tf
0     supreme  1.0
1     alabama  1.0
2        adam  1.0
3      horton  1.0
4      tanner  1.0
..        ...  ...
119   stewart  1.0
120    filing  2.0
121  negative  2.0
122    result  2.0
123   history  2.0

[124 rows x 2 columns]


In [ ]:
# top 10 n-grams 
from textblob import TextBlob
for i in range(11):
    print(TextBlob(train_new['data'][i]).ngrams(1))
    print(TextBlob(train_new['data'][i]).ngrams(2))
    print(TextBlob(train_new['data'][i]).ngrams(3))
    

[WordList(['supreme']), WordList(['alabama'])]
[WordList(['supreme', 'alabama'])]
[]
[WordList(['adam'])]
[]
[]
[WordList(['tanner']), WordList(['horton'])]
[WordList(['tanner', 'horton'])]
[]
[WordList(['synopsis'])]
[]
[]
[WordList(['writ']), WordList(['error']), WordList(['sumter'])]
[WordList(['writ', 'error']), WordList(['error', 'sumter'])]
[WordList(['writ', 'error', 'sumter'])]
[WordList(['west']), WordList(['headnotes'])]
[WordList(['west', 'headnotes'])]
[]
[WordList(['1'])]
[]
[]
[WordList(['chattel']), WordList(['mortgage'])]
[WordList(['chattel', 'mortgage'])]
[]
[WordList(['crop'])]
[]
[]
[WordList(['4']), WordList(['case']), WordList(['cite']), WordList(['headnote'])]
[WordList(['4', 'case']), WordList(['case', 'cite']), WordList(['cite', 'headnote'])]
[WordList(['4', 'case', 'cite']), WordList(['case', 'cite', 'headnote'])]
[WordList(['creditor']), WordList(['remedy'])]
[WordList(['creditor', 'remedy'])]
[]


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
postag = []
for sentence in train_new['data']:
  text = word_tokenize(str(sentence)) 
  postag.append(nltk.pos_tag(text)) # tagging
print(postag)
from collections import Counter # using counter
counts = []
for tags in postag:
  count = Counter( tag for word,  tag in tags)
  counts.append(count)
counts



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\preet\AppData\Roaming\nltk_data...


[[('supreme', 'NN'), ('alabama', 'NN')], [('adam', 'NN')], [('tanner', 'NN'), ('horton', 'NN')], [('synopsis', 'NN')], [('writ', 'NN'), ('error', 'NN'), ('sumter', 'NN')], [('west', 'NN'), ('headnotes', 'NNS')], [('1', 'CD')], [('chattel', 'NN'), ('mortgage', 'NN')], [('crop', 'NN')], [('4', 'CD'), ('case', 'NN'), ('cite', 'JJ'), ('headnote', 'NN')], [('creditor', 'NN'), ('remedy', 'NN')], [('lien', 'NN'), ('priority', 'NN')], [('5', 'CD'), ('case', 'NN'), ('cite', 'JJ'), ('headnote', 'NN')], [('attorney', 'NN'), ('law', 'NN'), ('firm', 'NN')], [('opinion', 'NN')], [('dissenting', 'VBG'), ('opinion', 'NN')], [('citation', 'NN')], [('end', 'NN')], [('2019', 'CD'), ('thomson', 'NN'), ('reuters', 'NNS'), ('original', 'JJ'), ('u', 'JJ'), ('government', 'NN'), ('work', 'NN')], [('citing', 'VBG'), ('reference', 'NN'), ('9', 'CD')], [('treatment', 'NN')], [('title', 'NN')], [('date', 'NN')], [('type', 'NN')], [('depth', 'NN')], [('headnotes', 'NNS')], [('1', 'CD'), ('booker', 'NN'), ('jones',

[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


[Counter({'NN': 2}),
 Counter({'NN': 1}),
 Counter({'NN': 2}),
 Counter({'NN': 1}),
 Counter({'NN': 3}),
 Counter({'NN': 1, 'NNS': 1}),
 Counter({'CD': 1}),
 Counter({'NN': 2}),
 Counter({'NN': 1}),
 Counter({'CD': 1, 'NN': 2, 'JJ': 1}),
 Counter({'NN': 2}),
 Counter({'NN': 2}),
 Counter({'CD': 1, 'NN': 2, 'JJ': 1}),
 Counter({'NN': 3}),
 Counter({'NN': 1}),
 Counter({'VBG': 1, 'NN': 1}),
 Counter({'NN': 1}),
 Counter({'NN': 1}),
 Counter({'CD': 1, 'NN': 3, 'NNS': 1, 'JJ': 2}),
 Counter({'VBG': 1, 'NN': 1, 'CD': 1}),
 Counter({'NN': 1}),
 Counter({'NN': 1}),
 Counter({'NN': 1}),
 Counter({'NN': 1}),
 Counter({'NN': 1}),
 Counter({'NNS': 1}),
 Counter({'CD': 1, 'NN': 1, 'NNS': 1, 'VBP': 1}),
 Counter({'NN': 1, 'CD': 1}),
 Counter({'NN': 7}),
 Counter({'NN': 1, 'CD': 1}),
 Counter({'CD': 1, 'NN': 2}),
 Counter({'NN': 7, 'JJ': 1}),
 Counter({'NN': 1, 'CD': 1}),
 Counter({'CD': 1, 'NN': 2}),
 Counter({'NN': 4, 'JJ': 1, 'CD': 1}),
 Counter({'NN': 1}),
 Counter({'CD': 1, 'NNS': 1, 'VBP': 1})

In [ ]:
import benepar
parser = benepar.Parser("benepar_en3") 
for sentence in train_new['data']:
    tree = parser.parse(sentence)
    print(tree)


C:\Users\preet\anaconda3\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


(TOP (NP (JJ supreme) (NN alabama)))
(TOP (X (UH adam)))
(TOP (NP (NN tanner) (NN horton)))
(TOP (NP (NN synopsis)))
(TOP (NP (NML (NN writ) (NN error)) (NN sumter)))
(TOP (X (NN west) (VP (VBZ headnotes))))
(TOP (NP (CD 1)))
(TOP (NP (NN chattel) (NN mortgage)))
(TOP (NP (NN crop)))
(TOP (X (NP (CD 4) (NN case)) (VP (VB cite) (NP (NN headnote)))))
(TOP (NP (NN creditor) (NN remedy)))
(TOP (NP (NN lien) (NN priority)))
(TOP (S (NP (CD 5) (NN case)) (VP (VB cite) (NP (NN headnote)))))
(TOP (NP (NN attorney) (NN law) (NN firm)))
(TOP (NP (NN opinion)))
(TOP (NP (JJ dissenting) (NN opinion)))
(TOP (NP (NN citation)))
(TOP (S (VP (VB end))))
(TOP
  (NP
    (CD 2019)
    (NN thomson)
    (NNS reuters)
    (JJ original)
    (FW u)
    (NN government)
    (NN work)))
(TOP (S (VP (VBG citing) (NP (NN reference) (CD 9)))))
(TOP (NP (NN treatment)))
(TOP (NP (NN title)))
(TOP (NP (NN date)))
(TOP (S (VP (VB type))))
(TOP (NP (NN depth)))
(TOP (NP (NNS headnotes)))
(TOP (S (NP (CD 1) (NN booker))

In [ ]:
import spacy 
from spacy import displacy

nlp = spacy.load('en_core_web_sm') 
options = {'compact': True, 'font': 'Source Sans Pro', 'distance': 100}
for sentence in train_new['text']:
  doc = nlp(sentence)
  displacy.render(doc, style = 'dep', options=options, jupyter=True) 

# Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2): 

In [ ]:
# You explanation here

constituency parsing:
The constituency parse tree is based on the formalism of context-free grammars. In this type of tree, the sentence is divided into constituents, that is, sub-phrases that belong to a specific category in the grammar.

In English, for example, the phrases “a dog”, “a computer on the table” and “the nice sunset” are all noun phrases, while “eat a pizza” and “go to the beach” are verb phrases.

dependency parsing: 
As opposed to constituency parsing, dependency parsing doesn’t make use of phrasal constituents or sub-phrases. Instead, the syntax of the sentence is expressed in terms of dependencies between words — that is, directed, typed edges between words in a graph.


In [ ]:
#Entity recognition 
import pandas as pd
train_new = pd.read_csv(r'C:\Users\preet\Desktop\5731\out.csv',index_col=0)
train_new = train_new.dropna()
train_new=train_new.reset_index()
del train_new['index']
train_new.head()
import en_core_web_sm 
nlp = en_core_web_sm.load()

for sentence in train_new['text']:
  documents = nlp(sentence)
  for X in documents.ents:
    if X.text and X.label_:
      print([(X.text, X.label_)])

[('supreme', 'ORG')]
[('tanner horton', 'PERSON')]
[('1', 'CARDINAL')]
[('4', 'CARDINAL')]
[('lien', 'PERSON')]
[('5', 'CARDINAL')]
[('reuters', 'ORG')]
[('9', 'CARDINAL')]
[('1', 'CARDINAL')]
[('jones admx', 'PERSON')]
[('dec 1876', 'DATE')]
[('john cunningham', 'PERSON')]
[('john cunningham', 'PERSON')]
[('jan 1871', 'DATE')]
[('jan 1858', 'DATE')]
[('5', 'CARDINAL')]
[('6', 'CARDINAL')]
[('1857', 'DATE')]
[('nov 1880', 'DATE')]
[('8', 'CARDINAL')]
[('edward thompson', 'PERSON')]
[('1887', 'DATE')]
[('9', 'CARDINAL')]
[('103', 'CARDINAL')]
[('464', 'CARDINAL')]
[('1936', 'DATE')]
[('3', 'CARDINAL')]
[('austin', 'PERSON')]


# **Question 4**

(20 points) Python Regular Expression.

(1) Write a Python program to remove leading zeros from an IP address.

ip = "260.08.094.109"


(2) Write a Python Program to extract all the years from the following sentence.

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [ ]:
# Write your code here

import re
ip = "260.08.094.109"
string = re.sub('\.[0]*', '.', ip)
print(string)

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."
lst=[] 
b=0 
s="" 

for i in sentence:
  if i=='2': 
    b=1
  if not(i.isdigit()) and b==1: 
    b=0
  if i.isdigit() and b==1: 
    s+=i
  if s!="" and i==" ": 
    lst.append(int(s)) 
    s="" 
print(lst)




260.8.94.109
[2010, 2010, 2019]
